In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd
import wget
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


In [4]:
# download mediapipe model bundle
wget.download("https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task")

'hand_landmarker.task'

In [2]:
# Google's pre-written function to visualise hand landmark detection image
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [7]:
# function to read in all images in a folder
def read_images_from_folder(folder_path):
    """
    Reads all images from the specified folder and stores them in a dictionary.

    Args:
        folder_path (str): The path to the folder containing the images.

    Returns:
        dict: A dictionary where keys are filenames and values are the images.
    """
    images = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.tif', '.PNG')):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images[filename] = img
            else:
                print(f"Warning: {filename} could not be read as an image.")
    return images

In [8]:
# read in all images from images folder using new function
folder_path='C:/Users/awright/Documents/git_repos/bsl-interpreter/images/youtube_video'
bsl_images = read_images_from_folder(folder_path)

In [11]:
# get lists of keys and values for images dictionary
bsl_images_keys = list(bsl_images)
bsl_images_vals = list(bsl_images.values())
bsl_images_vals

[array([[[108,  90, 105],
         [108,  90, 105],
         [108,  90, 105],
         ...,
         [ 61,  51,  65],
         [ 60,  50,  64],
         [ 60,  50,  64]],
 
        [[108,  90, 105],
         [108,  90, 105],
         [108,  90, 105],
         ...,
         [ 61,  51,  65],
         [ 60,  50,  64],
         [ 60,  50,  64]],
 
        [[108,  90, 105],
         [108,  90, 105],
         [108,  90, 105],
         ...,
         [ 61,  51,  65],
         [ 60,  50,  64],
         [ 60,  50,  64]],
 
        ...,
 
        [[173, 174, 195],
         [173, 174, 196],
         [174, 175, 196],
         ...,
         [ 86,  84,  97],
         [ 84,  84,  97],
         [ 84,  83,  97]],
 
        [[173, 174, 195],
         [174, 174, 195],
         [174, 175, 195],
         ...,
         [ 87,  86,  99],
         [ 85,  84,  98],
         [ 85,  84,  98]],
 
        [[174, 174, 195],
         [174, 174, 195],
         [174, 175, 195],
         ...,
         [ 88,  87, 100],
  

In [ ]:
# test import of images
#cv2.namedWindow('frame', cv2.WINDOW_AUTOSIZE)
#cv2.imshow('frame', bsl_images['bsl_a_2.png'])
#cv2.waitKey(0)
# only run if necessary - notebooks don't like showing the images!!! 
# can cause kernel to crash

97

: 

In [66]:
# Create HandLandmarker object
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2,
                                       min_hand_detection_confidence=0.01, # very low threshold
                                       min_hand_presence_confidence=0.01
                                       )
detector = vision.HandLandmarker.create_from_options(options)

In [65]:
# Load input image
image = mp.Image(image_format=mp.ImageFormat.SRGB, data=bsl_images_vals[15])

In [69]:
# Detect hand landmarks from input image
detection_result = detector.detect(image)

In [70]:
im2 = image.numpy_view()
copied_im2_array = np.copy(im2)


In [71]:
print(copied_im2_array.shape)
type(copied_im2_array)

(576, 711, 3)


numpy.ndarray

In [72]:
bgr_img = copied_im2_array[:,:,0:3]

In [73]:
# Process classification result - visualise it
annotated_image = draw_landmarks_on_image(bgr_img, detection_result)

In [74]:
cv2.imshow('frame', annotated_image)
cv2.waitKey(0)

-1

In [22]:
x = annotated_image - bgr_img
cv2.imshow('frame', x)
cv2.waitKey(0)
# not reading the hands in the image as hands - need to fine tune model

-1